# COVID-19 - Corona Virus Analysis 
### Context
> From World Health Organization - On 31 December 2019, WHO was alerted to several cases of pneumonia in Wuhan City, Hubei Province of China. The virus did not match any other known virus. This raised concern because when a virus is new, we do not know how it affects people.
> 
> So daily level information on the affected people can give some interesting insights when it is made available to the broader data science community.
> 
> Johns Hopkins University has made an excellent dashboard using the affected cases data. This data is extracted from the same link and made available in csv format.

In [ ]:
import pandas as pd,\
numpy as np,\
matplotlib.pyplot as plt,\
seaborn as sns,\
plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

### Import Data
Simple preprocessing

**NOTE: Diamond Princess is changed to DP**

In [ ]:
nCov = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv", 
                   usecols=['ObservationDate', 'Province/State', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered'])

nCov.columns = ['Date', 'Province/State', 'Country', 'Confirmed', 'Deaths', 'Recovered']

nCov['Date'] = pd.to_datetime(nCov['Date']).dt.to_period('D')
nCov['Date'] = nCov['Date'].values.astype('datetime64[D]')
# nCov['Province/State'] = nCov['Province/State'].str.replace('\(From Diamond Princess\)', "")
nCov['Province/State'] = nCov['Province/State'].str.replace('Diamond Princess', "DP")
nCov['Province/State'] = nCov['Province/State'].str.replace('From DP', "DP")
nCov['Country'] = nCov['Country'].str.replace('Diamond Princess', "DP")

nCov.replace('None', np.nan, inplace=True)
nCov.fillna('Not Specified', inplace=True)

In [ ]:
country_code = pd.read_csv('../input/plotly_countries/2014_world_gdp_with_codes.csv', usecols=['COUNTRY', 'CODE'])
country_code.loc[country_code['COUNTRY'] == "Korea, South", 'COUNTRY'] = 'South Korea'
country_code.loc[country_code['COUNTRY'] == "United States", 'COUNTRY'] = 'US'
country_code.loc[country_code['COUNTRY'] == "United Kingdom", 'COUNTRY'] = 'UK'
country_code.loc[country_code['COUNTRY'] == "Cote d'Ivoire", 'COUNTRY'] = "Ivory Coast"
country_code.loc[country_code['COUNTRY'] == "Korea, South", 'COUNTRY'] = 'South Korea'
country_code.loc[country_code['COUNTRY'] == "China", 'COUNTRY'] = 'Mainland China'

_Country names edited for dataset same value matching_

### COVID-19 Dataset

In [ ]:
nCov.head()

### Country Code  Dataset

In [ ]:
country_code.head()

# Descriptive Statistics

In [ ]:
nCov_describe = nCov.describe()
nCov_describe

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "bar"}, {"type": "bar"}],
          [{"type": "bar"}, {"type": "bar"}]], 
    subplot_titles=("Average", "Standard Deviation", "Minimum", "Maximum")
)

nCov_mean = nCov_describe.loc['mean'].sort_values(ascending=False)
nCov_std = nCov_describe.loc['std'].sort_values(ascending=False)
nCov_min = nCov_describe.loc['min'].sort_values(ascending=False)
nCov_max = nCov_describe.loc['max'].sort_values(ascending=False)

fig.add_trace(go.Bar(y=nCov_mean, x=nCov_mean.index, text=nCov_mean, textposition='auto',
                     marker={'color': ['orange', 'green', 'red'], 'opacity': 0.75}, 
                     name='Average'), 1, 1)

fig.add_trace(go.Bar(y=nCov_std, x=nCov_std.index, text=nCov_std, textposition='auto',
                     marker={'color': ['orange', 'green', 'red'], 'opacity': 0.75}, 
                     name='Standard Deviation'), 1, 2)

fig.add_trace(go.Bar(y=nCov_min, x=nCov_min.index, text=nCov_min, textposition='auto',
                     marker={'color': ['orange', 'green', 'red'], 'opacity': 0.75}, 
                     name='Minimum'), 2, 1)

fig.add_trace(go.Bar(y=nCov_max, x=nCov_max.index, text=nCov_max, textposition='auto',
                     marker={'color': ['orange', 'green', 'red'], 'opacity': 0.75}, 
                     name='Maximum'),2,2)

fig.update_layout(height=750, showlegend=False, width=750,
                  title_text='Global COVID-19 Case Status Statistical Properties per Province, 2020', uniformtext_minsize=4)
fig.update_traces(texttemplate='%{text:.1f}')

fig.show()

# Preprocessing

### Country Grouping
_Note that cases are **cumulative sum**, thus we need to get the last date for each province, then group sum by country_

In [ ]:
country_grouped = nCov.groupby(['Country', 'Province/State']).max().groupby('Country').sum()
country_grouped['Country'] = country_grouped.index
country_grouped.head()

### Province Grouping

In [ ]:
province_grouped = nCov.groupby(['Country', 'Province/State']).max()
province_grouped['Country'] = province_grouped.index.get_level_values(0)
province_grouped['Province/State'] = province_grouped.index.get_level_values(1)
province_grouped = province_grouped.droplevel(0).drop('Date', axis=1)
province_grouped.head()

### Two Date Groupings - Cumulative Sum and actual values by day
_**REMEMBER:** Any immediate aggregation by date is wrong, numbers won't represent real values but summed up cumulative sums_

**STEPS:**
1. Group by Country, Province, Date (No function is applied)
2. get the difference between each two proceeding values in groups
3. Fix first values
4. Get the actual **Grouping Sums then from that, cumsum**

In [ ]:
date_grouped = nCov.groupby(['Country', 'Province/State', 'Date']).max() # Grouping is only to sort. "Max" doesn't do anything
date_grouped = date_grouped.diff(axis=0)
date_grouped.\
loc[nCov.groupby(['Country', 'Province/State']).nth(0).\
set_index('Date', append=True).index] = \
nCov.groupby(['Country', 'Province/State']).nth(0)\
.set_index('Date', append=True)

In [ ]:
date_grouped = nCov.groupby(['Country', 'Province/State', 'Date']).max() # Grouping is only to sort. "Max" doesn't do anything
date_grouped = date_grouped.diff(axis=0)
date_grouped.\
loc[nCov.groupby(['Country', 'Province/State']).nth(0).\
set_index('Date', append=True).index] = \
nCov.groupby(['Country', 'Province/State']).nth(0)\
.set_index('Date', append=True)
date_grouped.loc[date_grouped['Recovered'] < 0, ['Recovered']] = 0
date_grouped.loc[date_grouped['Confirmed'] < 0, ['Confirmed']] = 0
date_grouped.loc[date_grouped['Deaths'] < 0, ['Deaths']] = 0

In [ ]:
date_country_grouped = date_grouped

In [ ]:
date_grouped_sum = date_grouped.groupby(['Date']).sum()
date_grouped_cumsum = date_grouped_sum.cumsum()
date_grouped_sum['Date'] = date_grouped_sum.index
date_grouped_cumsum['Date'] = date_grouped_cumsum.index

In [ ]:
date_grouped_sum.head()

In [ ]:
date_grouped_cumsum.head()

In [ ]:
df = country_grouped\
.merge(country_code.set_index('COUNTRY'), left_index=True, right_index=True, how='left').dropna()

# COVID-19 Spread in Countries

In [ ]:
## CONFIRMED
fig = px.choropleth(df, locations="CODE",
                    color="Confirmed",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Peach, range_color=[0, 40], labels={"Confirmed": "Scaled Confirmed"})

fig.update_layout(title_text="Global COVID-19 Confirmed Cases")
fig.show()

## RECOVERED
fig = px.choropleth(df, locations="CODE",
                    color="Recovered",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Viridis, range_color=[1,40], labels={"Recovered": "Scaled Recovered"})

fig.update_layout(title_text="Global COVID-19 Recovered Cases")
fig.show()

## DEATHS
fig = px.choropleth(df, locations="CODE",
                    color="Deaths",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Inferno, range_color=[1,40], labels={"Deaths": "Scaled Deaths"})

fig.update_layout(title_text="Global COVID-19 Death Cases")
fig.show()

# Proportion of Cases in Countries & Provinces

In [ ]:
# pg_tree = province_grouped.copy()

pg_tree = nCov.copy()
pg_tree['Under Treatment'] = pg_tree['Confirmed'] - (pg_tree['Recovered'] + pg_tree['Deaths'])
pg_tree.drop('Confirmed', axis=1, inplace=True)
pg_tree = pd.DataFrame(pg_tree.groupby(['Country', 'Province/State']).max().drop('Date', axis=1).stack(), columns=['Value'])
pg_tree['Country'] = pg_tree.index.get_level_values(0)
pg_tree['Province/State'] = pg_tree.index.get_level_values(1)
pg_tree['Status'] = pg_tree.index.get_level_values(2)
pg_tree.droplevel([0, 1])
pg_tree.reset_index(drop=True, inplace=True)

world_sun = pg_tree
world_sun_exc_china = pg_tree.loc[pg_tree['Country'] != 'Mainland China']

world_sun_exc = pg_tree.loc[(pg_tree['Country'] != 'South Korea') &
                            (pg_tree['Country'] != 'Mainland China') &
                                 (pg_tree['Country'] != 'Italy') &
                                 (pg_tree['Country'] != 'Others') &
                                 (pg_tree['Country'] != 'Iran')]

china_sun = pg_tree.loc[pg_tree['Country'] == 'Mainland China'].drop('Country', axis=1)
china_sun_exec_hubei = china_sun.drop(china_sun.loc[china_sun['Province/State'] == 'Hubei'].index)
us_sun = pg_tree.loc[pg_tree['Country'] == 'US'].drop('Country', axis=1)



fig = px.sunburst(world_sun.replace(0, 0.0000001), values='Value', maxdepth=3,
                  path=['Country', 'Province/State', 'Status'])
fig.update_layout(height=800,uniformtext=dict(minsize=9 ,mode='hide'), title_text="Worldwide Cases")
fig.show()

fig = px.sunburst(world_sun_exc_china.replace(0, 0.0000001), values='Value', maxdepth=3,
                  path=['Country', 'Province/State', 'Status'])
fig.update_layout(height=800,uniformtext=dict(minsize=9 ,mode='hide'), title_text="Worldwide Cases (Exclusion: China)")
fig.show()

fig = px.sunburst(world_sun_exc.replace(0, 0.0000001), values='Value', maxdepth=3,
                  path=['Country', 'Province/State', 'Status'])
fig.update_layout(height=800,uniformtext=dict(minsize=9 ,mode='hide'), title_text="Worldwide Cases (Exclusion: China, S-Korea, Italy, Others, Iran)")
fig.show()

fig = px.sunburst(china_sun.replace(0, 0.0000001), values='Value', maxdepth=3,
                  path=[ 'Province/State', 'Status'])
fig.update_layout(height=800,uniformtext=dict(minsize=9 ,mode='hide'), title_text="China Cases")
fig.show()

fig = px.sunburst(china_sun_exec_hubei.replace(0, 0.0000001), values='Value', maxdepth=3,
                  path=['Province/State', 'Status'])
fig.update_layout(height=800,uniformtext=dict(minsize=9 ,mode='hide'), title_text="China Cases (Exclusion: Hubei)")
fig.show()

fig = px.sunburst(us_sun.replace(0, 0.0000001), values='Value', maxdepth=3,
                  path=['Province/State', 'Status'])
fig.update_layout(height=800,uniformtext=dict(minsize=9 ,mode='hide'), title_text="US Cases")
fig.show()

# Percentage of Types of Cases in Countries & Provinces

In [ ]:
pie_chart_cases = country_grouped.copy()
pie_chart_cases['Under Treatment'] = pie_chart_cases['Confirmed'] - (pie_chart_cases['Deaths'] + pie_chart_cases['Recovered'])
pie_chart_cases.drop('Confirmed', axis = 1, inplace = True)
pie_chart_cases = pie_chart_cases[['Under Treatment', 'Deaths', 'Recovered']].sum()

fig = go.Figure()

fig.add_trace(go.Pie(labels=pie_chart_cases.index, text=pie_chart_cases.values,
                     values=pie_chart_cases.values, name="Cases", marker={'colors': ['orange', 'red', 'rgb(0,204,150)']}))

fig.update_traces(hoverinfo="label+percent+name", textposition='inside', insidetextfont=dict(size=16))
fig.update_layout(height=700, uniformtext_mode=False, 
    title_text="Global COVID-19 Cases")
fig.show()

In [ ]:
country_grouped_pie = country_grouped

country_grouped_conf = country_grouped_pie.drop(country_grouped_pie.loc[country_grouped_pie['Confirmed'] == 0].index)
country_grouped_rec = country_grouped_pie.drop(country_grouped_pie.loc[country_grouped_pie['Recovered'] == 0].index)
country_grouped_death = country_grouped_pie.drop(country_grouped_pie.loc[country_grouped_pie['Deaths'] == 0].index)

fig = make_subplots(rows=2, cols=2, column_width=[0.65, 0.35], vertical_spacing = 0.1,
                    specs=[[{'type':'domain', "colspan": 2}, None], 
                           [{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=country_grouped_conf.index, 
                     values=country_grouped_conf['Confirmed'], name="Confirmed Cases"), 1, 1)

fig.add_trace(go.Pie(labels=country_grouped_rec.index, 
                     values=country_grouped_rec['Recovered'], name="Recovered Cases"), 2, 1)

fig.add_trace(go.Pie(labels=country_grouped_death.index, 
                     values=country_grouped_death['Deaths'], name="Death Cases"), 2, 2)

fig.update_traces(hoverinfo="label+percent+name", textposition='inside')
fig.update_layout(height=1200, uniformtext_mode=False,
    title_text=" COVID-19 Cases in All Countries",
    annotations=[{'text':'Confirmed', 'x':0.1, 'y':1, 'font_size':18, 'showarrow':False},
                 {'text':'Recovered', 'x':-0.1, 'y':0.4, 'font_size':18, 'showarrow':False},
                 {'text':'Deaths', 'x':0.75, 'y':0.35, 'font_size':18, 'showarrow':False}])
fig.show()

##########

country_grouped_pie = country_grouped.drop('Mainland China')

country_grouped_conf = country_grouped_pie.drop(country_grouped_pie.loc[country_grouped_pie['Confirmed'] == 0].index)
country_grouped_rec = country_grouped_pie.drop(country_grouped_pie.loc[country_grouped_pie['Recovered'] == 0].index)
country_grouped_death = country_grouped_pie.drop(country_grouped_pie.loc[country_grouped_pie['Deaths'] == 0].index)

fig = make_subplots(rows=2, cols=2, column_width=[0.65, 0.35], vertical_spacing = 0.1,
                    specs=[[{'type':'domain', "colspan": 2}, None], 
                           [{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=country_grouped_conf.index, 
                     values=country_grouped_conf['Confirmed'], name="Confirmed Cases"), 1, 1)

fig.add_trace(go.Pie(labels=country_grouped_rec.index, 
                     values=country_grouped_rec['Recovered'], name="Recovered Cases"), 2, 1)

fig.add_trace(go.Pie(labels=country_grouped_death.index, 
                     values=country_grouped_death['Deaths'], name="Death Cases"), 2, 2)

fig.update_traces(hoverinfo="label+percent+name", textposition='inside')
fig.update_layout(height=1200, uniformtext_mode=False,
    title_text="COVID-19 Cases in All Countries (Exclusion: China)",
    annotations=[{'text':'Confirmed', 'x':0.1, 'y':1, 'font_size':18, 'showarrow':False},
                 {'text':'Recovered', 'x':-0.1, 'y':0.4, 'font_size':18, 'showarrow':False},
                 {'text':'Deaths', 'x':0.75, 'y':0.35, 'font_size':18, 'showarrow':False}])
fig.show()
#######################



province_grouped_pie = province_grouped

country_grouped_conf = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Confirmed'] == 0].index)
country_grouped_rec = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Recovered'] == 0].index)
country_grouped_death = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Deaths'] == 0].index)

fig = make_subplots(rows=2, cols=2, column_width=[0.65, 0.35], vertical_spacing = 0.1,
                    specs=[[{'type':'domain', "colspan": 2}, None], 
                           [{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=country_grouped_conf.index, 
                     values=country_grouped_conf['Confirmed'], name="Confirmed Cases"), 1, 1)

fig.add_trace(go.Pie(labels=country_grouped_rec.index, 
                     values=country_grouped_rec['Recovered'], name="Recovered Cases"), 2, 1)

fig.add_trace(go.Pie(labels=country_grouped_death.index, 
                     values=country_grouped_death['Deaths'], name="Death Cases"), 2, 2)

fig.update_traces(hoverinfo="label+percent+name", textposition='inside')
fig.update_layout(height=1200, uniformtext_mode=False,
    title_text=" COVID-19 Cases in All Provinces",
    annotations=[{'text':'Confirmed', 'x':0.1, 'y':1, 'font_size':18, 'showarrow':False},
                 {'text':'Recovered', 'x':-0.1, 'y':0.4, 'font_size':18, 'showarrow':False},
                 {'text':'Deaths', 'x':0.75, 'y':0.35, 'font_size':18, 'showarrow':False}])
fig.show()

##########

province_grouped_pie = province_grouped.drop('Hubei')

country_grouped_conf = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Confirmed'] == 0].index)
country_grouped_rec = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Recovered'] == 0].index)
country_grouped_death = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Deaths'] == 0].index)

fig = make_subplots(rows=2, cols=2, column_width=[0.65, 0.35], vertical_spacing = 0.1,
                    specs=[[{'type':'domain', "colspan": 2}, None], 
                           [{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=country_grouped_conf.index, 
                     values=country_grouped_conf['Confirmed'], name="Confirmed Cases"), 1, 1)

fig.add_trace(go.Pie(labels=country_grouped_rec.index, 
                     values=country_grouped_rec['Recovered'], name="Recovered Cases"), 2, 1)

fig.add_trace(go.Pie(labels=country_grouped_death.index, 
                     values=country_grouped_death['Deaths'], name="Death Cases"), 2, 2)

fig.update_traces(hoverinfo="label+percent+name", textposition='inside')
fig.update_layout(height=1200, uniformtext_mode=False,
    title_text="COVID-19 Cases in All Provinces (Exclusion: Hubei)",
    annotations=[{'text':'Confirmed', 'x':0.1, 'y':1, 'font_size':18, 'showarrow':False},
                 {'text':'Recovered', 'x':-0.1, 'y':0.4, 'font_size':18, 'showarrow':False},
                 {'text':'Deaths', 'x':0.75, 'y':0.35, 'font_size':18, 'showarrow':False}])
fig.show()

In [ ]:
province_grouped_pie = province_grouped[province_grouped['Country'] == 'Mainland China']

country_grouped_conf = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Confirmed'] == 0].index)
country_grouped_rec = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Recovered'] == 0].index)
country_grouped_death = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Deaths'] == 0].index)

fig = make_subplots(rows=2, cols=2, column_width=[0.65, 0.35], vertical_spacing = 0.1,
                    specs=[[{'type':'domain', "colspan": 2}, None], 
                           [{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=country_grouped_conf.index, 
                     values=country_grouped_conf['Confirmed'], name="Confirmed Cases"), 1, 1)

fig.add_trace(go.Pie(labels=country_grouped_rec.index, 
                     values=country_grouped_rec['Recovered'], name="Recovered Cases"), 2, 1)

fig.add_trace(go.Pie(labels=country_grouped_death.index, 
                     values=country_grouped_death['Deaths'], name="Death Cases"), 2, 2)

fig.update_traces(hoverinfo="label+percent+name", textposition='inside')
fig.update_layout(height=1200, uniformtext_mode=False,
    title_text=" China COVID-19 Cases in Provinces",
    annotations=[{'text':'Confirmed', 'x':0.1, 'y':1, 'font_size':18, 'showarrow':False},
                 {'text':'Recovered', 'x':-0.1, 'y':0.4, 'font_size':18, 'showarrow':False},
                 {'text':'Deaths', 'x':0.75, 'y':0.35, 'font_size':18, 'showarrow':False}])
fig.show()

##########

province_grouped_pie = province_grouped[province_grouped['Country'] == 'Mainland China'].drop('Hubei')

country_grouped_conf = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Confirmed'] == 0].index)
country_grouped_rec = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Recovered'] == 0].index)
country_grouped_death = province_grouped_pie.drop(province_grouped_pie.loc[province_grouped_pie['Deaths'] == 0].index)

fig = make_subplots(rows=2, cols=2, column_width=[0.65, 0.35], vertical_spacing = 0.1,
                    specs=[[{'type':'domain', "colspan": 2}, None], 
                           [{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=country_grouped_conf.index, 
                     values=country_grouped_conf['Confirmed'], name="Confirmed Cases"), 1, 1)

fig.add_trace(go.Pie(labels=country_grouped_rec.index, 
                     values=country_grouped_rec['Recovered'], name="Recovered Cases"), 2, 1)

fig.add_trace(go.Pie(labels=country_grouped_death.index, 
                     values=country_grouped_death['Deaths'], name="Death Cases"), 2, 2)

fig.update_traces(hoverinfo="label+percent+name", textposition='inside')
fig.update_layout(height=1200, uniformtext_mode=False,
    title_text=" China COVID-19 Cases in Provinces (Exclusion: Hubei)",
    annotations=[{'text':'Confirmed', 'x':0.1, 'y':1, 'font_size':18, 'showarrow':False},
                 {'text':'Recovered', 'x':-0.1, 'y':0.4, 'font_size':18, 'showarrow':False},
                 {'text':'Deaths', 'x':0.75, 'y':0.35, 'font_size':18, 'showarrow':False}])
fig.show()

# Detailed Case Types by Country

In [ ]:
country_grouped_exc_mainland = country_grouped.drop(['Mainland China']).sort_values(by='Confirmed')
fig = px.bar(country_grouped_exc_mainland, x="Confirmed", y="Country",
             color="Deaths", text='Confirmed',
             title='Confirmed & Death Cases in Countries - (Exclusion: China)', labels={"Confirmed": "Confirmed Cases"},
             height=1500, orientation='h')
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=12, yaxis=dict(tickangle=10, tickmode='linear'))
fig.show()

In [ ]:
country_grouped_exc_mainland.sort_values(by='Recovered', ascending=True, inplace=True)

fig = px.bar(country_grouped_exc_mainland, x="Recovered", y="Country",
             color="Deaths", text='Recovered',
             title='Recovered & Death Cases in Countries (Exclusion: China)', labels={"Recovered": "Recovered Cases"},
             height=1500, orientation='h')
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=12, yaxis=dict(tickangle=10, tickmode='linear'))
fig.show()

###  Detailed Case Types in Mainland China

In [ ]:
china_provinces = date_country_grouped.loc['Mainland China'].groupby('Province/State').max()\
.sort_values(by='Confirmed')


fig = go.Figure()
fig.add_trace(go.Bar(x=china_provinces['Confirmed'], y=china_provinces.index.tolist(), 
                     orientation='h', text= china_provinces['Confirmed'], 
                    name='Confirmed', textposition='none', marker={'color': 'orange', 'opacity': 0.75}))

fig.add_trace(go.Bar(x=china_provinces['Recovered'], y=china_provinces.index.tolist(),
                     orientation='h', text= china_provinces['Recovered'],
                    name='Recovered', textposition='none', marker={'color': 'green', 'opacity': 0.75}))

fig.add_trace(go.Bar(x=china_provinces['Deaths'], y=china_provinces.index.tolist(),
                      orientation='h', text= china_provinces['Deaths'],
                    name='Deaths', textposition='outside', marker={'color': 'red', 'opacity': 0.75}))


fig.update_layout(height=700, showlegend=True, 
                  title_text='Cases in Provinces/Cities (Anotated by Death Cases)', uniformtext_minsize=12, barmode='stack')

fig.update_traces(texttemplate='%{text:.2d}')

fig.show()

china_provinces = date_country_grouped.loc['Mainland China'].drop('Hubei').groupby('Province/State').max()\
.sort_values(by='Confirmed')


fig = go.Figure()
fig.add_trace(go.Bar(x=china_provinces['Confirmed'], y=china_provinces.index.tolist(), 
                     orientation='h', text= china_provinces['Confirmed'], 
                    name='Confirmed', textposition='none', marker={'color': 'orange', 'opacity': 0.75}))

fig.add_trace(go.Bar(x=china_provinces['Recovered'], y=china_provinces.index.tolist(),
                     orientation='h', text= china_provinces['Recovered'],
                    name='Recovered', textposition='none', marker={'color': 'green', 'opacity': 0.75}))

fig.add_trace(go.Bar(x=china_provinces['Deaths'], y=china_provinces.index.tolist(),
                      orientation='h', text= china_provinces['Deaths'],
                    name='Deaths', textposition='outside', marker={'color': 'red', 'opacity': 0.75}))


fig.update_layout(height=700, showlegend=True, 
                  title_text='Cases in Provinces/Cities (Anotated by Death Cases, Exclusion: Hubei)', uniformtext_minsize=12, barmode='stack')

fig.update_traces(texttemplate='%{text:.2d}')

fig.show()

# COVID-19 Spread Time Series

In [ ]:
date_grouped = date_grouped_cumsum
date_grouped = date_grouped.sort_index(ascending=False)

fig = go.Figure()


fig.add_trace(go.Scatter(x=date_grouped.index.tolist(), y=date_grouped['Confirmed'],
                    mode='lines+markers',
                    name='Confirmed', marker={'color': 'orange', 'opacity': 0.8} ))
fig.add_trace(go.Scatter(x=date_grouped.index.tolist(), y=date_grouped['Recovered'],
                    mode='lines+markers',
                    name='Recovered', marker={'color': 'green', 'opacity': 0.6} ))
fig.add_trace(go.Scatter(x=date_grouped.index.tolist(), y=date_grouped['Deaths'],
                    mode='lines+markers',
                    name='Deaths', marker={'color': 'red', 'opacity': 1} ))

fig.update_layout(height=500, title_text="Global Corona Spread with Time (Cumulative)")
fig.show()

###############

fig = go.Figure()

fig.add_trace(go.Bar(y=date_grouped.index.tolist(), x=date_grouped['Confirmed'],orientation='h',
                    name='Confirmed', marker={'color': 'orange', 'opacity': 0.8} ))

fig.add_trace(go.Bar(y=date_grouped.index.tolist(), x=date_grouped['Recovered'],orientation='h',
                    name='Recovered', marker={'color': 'green', 'opacity': 0.6}))

fig.add_trace(go.Bar(y=date_grouped.index.tolist(), x=date_grouped['Deaths'],orientation='h',
                    name='Deaths', marker={'color': 'red', 'opacity': 1}))

fig.update_layout(barmode='stack')
fig.update_layout(height=900, title_text="Global Corona Spread with Time (Cumulative Stacked Vars)",
                  yaxis=dict(tickmode='linear', tickangle=30) )
fig.show()


In [ ]:
fig = px.bar(date_grouped_cumsum, x='Confirmed', y='Date', text='Confirmed',
              color='Deaths', hover_data=['Recovered'],
             labels={'Confirmed':'Confirmed Cases'}, height=900, 
             title='COVID-19 Confirmed and Death Cases with Time (Cumulative)', orientation='h')

fig.update_traces(texttemplate='%{text:.3s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, yaxis=dict(tickangle=20, tickmode='linear'))
fig.show()

fig = px.bar(date_grouped_sum, x='Confirmed', y='Date', text='Confirmed',
              color='Deaths', hover_data=['Recovered'],
             labels={'Confirmed':'Confirmed Cases'}, height=900, 
             title='COVID-19 Confirmed and Death Cases with Time (NON-Cumulative)', orientation='h')

fig.update_traces(texttemplate='%{text:.3s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, yaxis=dict(tickangle=20, tickmode='linear'))

fig.show()

# Country/Province-Specific Cases Time-Series

In [ ]:
china_date = date_country_grouped.loc['Mainland China'].drop('Hubei').droplevel(0).groupby('Date').sum().cumsum()
hubei_date = date_country_grouped.loc['Mainland China'].loc['Hubei'].groupby('Date').max().cumsum()

fig = go.Figure()
fig.add_trace(go.Scatter(x=china_date.index.tolist(), y=china_date['Confirmed'], mode="markers+lines",
                    name='Confirmed', textposition='top left', marker={'color': 'orange', 'opacity': 0.75}))

fig.add_trace(go.Scatter(x=china_date.index.tolist(), y=china_date['Recovered'], mode="markers+lines",
                    name='Recovered', textposition='top left', marker={'color': 'green', 'opacity': 0.75}))

fig.add_trace(go.Scatter(x=china_date.index.tolist(), y=china_date['Deaths'], mode="markers+lines",
                    name='Deaths', textposition='top left', marker={'color': 'red', 'opacity': 0.75}))
fig.update_layout(showlegend=True, 
                  title_text='Cases in Provinces/Cities with Time (Exclusion: Hubei, Cumulative)', uniformtext_minsize=12)

fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=hubei_date.index.tolist(), y=hubei_date['Confirmed'], mode="markers+lines",
                    name='Confirmed', marker={'color': 'orange', 'opacity': 0.75}))

fig.add_trace(go.Scatter(x=hubei_date.index.tolist(), y=hubei_date['Recovered'], mode="markers+lines",
                    name='Recovered', marker={'color': 'green', 'opacity': 0.75}))

fig.add_trace(go.Scatter(x=hubei_date.index.tolist(), y=hubei_date['Deaths'], mode="markers+lines",
                    name='Deaths', marker={'color': 'red', 'opacity': 0.75}))
fig.update_layout(showlegend=True, 
                  title_text='Cases in Hubei Province - Cumulative', uniformtext_minsize=12)

fig.show()


### Plots ordered by highest number of confirmed cases in countries

In [ ]:
def plot_time_scatter(date_country_grouped, country_name):
    country_date = date_country_grouped.loc[country_name].droplevel(0).groupby('Date').sum()
    country_date['Under Treatment'] = country_date['Confirmed'] - (country_date['Recovered'] + country_date['Deaths'])
    
    country_date[['Confirmed', 'Deaths', 'Recovered', 'Under Treatment']] = \
    country_date[['Confirmed', 'Deaths', 'Recovered', 'Under Treatment']].apply(np.abs)

    fig = go.Figure()
 
    fig.add_trace(go.Scatter(x=country_date.index.tolist(), y=country_date['Recovered'], mode="markers+lines",
                        name='Recovered', textposition='top left', marker={'color': 'green', 'opacity': 0.75}))

    fig.add_trace(go.Scatter(x=country_date.index.tolist(), y=country_date['Deaths'], mode="markers+lines",
                        name='Deaths', textposition='top left', marker={'color': 'red', 'opacity': 0.75}))

    fig.add_trace(go.Scatter(x=country_date.index.tolist(), y=country_date['Under Treatment'], mode="markers+lines",
                        name='Under Treatment', textposition='top left', marker={'color': 'gray', 'opacity': 0.75}))

    fig.update_layout(height=600, showlegend=True, 
                      title_text= ''+str(country_name)+ ' Cases with Time', uniformtext_minsize=12)

    fig.show()


plot_time_scatter(date_country_grouped, 'Mainland China')
plot_time_scatter(date_country_grouped, 'South Korea')
plot_time_scatter(date_country_grouped, 'Italy')
plot_time_scatter(date_country_grouped, 'Iran')
plot_time_scatter(date_country_grouped, 'Japan')
plot_time_scatter(date_country_grouped, 'US')
plot_time_scatter(date_country_grouped, 'Singapore')
plot_time_scatter(date_country_grouped, 'France')
plot_time_scatter(date_country_grouped, 'Hong Kong') 

# China and the Rest Time-Series

In [ ]:

mainland_china_dates = date_country_grouped.loc['Mainland China'].droplevel(0).groupby('Date').sum().cumsum()
rest_dates = date_country_grouped.drop('Mainland China').droplevel(0).groupby('Date').sum().cumsum()

fig = go.Figure()
fig.add_trace(go.Bar(y=mainland_china_dates.index,
                x=mainland_china_dates['Confirmed'],
                name='Mainland China', orientation='h',text=mainland_china_dates['Confirmed'],textposition='outside',
                marker_color='rgba(255, 0,0, 0.5)'
                ))
fig.add_trace(go.Bar(y=rest_dates.index,
                x=rest_dates['Confirmed'], text=rest_dates['Confirmed'], textposition='outside',
                name='Rest', orientation='h',
                marker_color='rgba(255,200,0, 1)'
                ))
fig.update_layout(height=1200, showlegend=True,
                      title_text= 'Cases with Time', uniformtext_minsize=12, yaxis=dict(tickmode='linear', tickangle=10))

# Ratios 
### (For countries exceeding confirmed cases median)

\begin{equation*}
\frac{Under Treatment}{Under Treatment + Deaths + Recovered},,, \frac{Deaths}{Under Treatment + Deaths + Recovered},,,\frac{Recovered}{Under Treatment + Deaths + Recovered}
\end{equation*}

In [ ]:
ratios = country_grouped

ratios['Under Treatment'] = ratios['Confirmed'] - (ratios['Recovered'] + ratios['Deaths'])

    
ratios = ratios[ratios['Confirmed'] > ratios['Confirmed'].median()][['Confirmed', 'Under Treatment', 'Recovered', 'Deaths']]
ratios = ratios.drop('Confirmed', axis=1)
ratios = (ratios.T / ratios.sum(axis=1)).T.dropna()

highest_death_ratio = ratios.sort_values(by='Deaths', ascending=False).iloc[:5,:]
highest_recovered_ratio = ratios.sort_values(by='Recovered', ascending=False).iloc[:5,:]
highest_ut_ratio = ratios.sort_values(by='Under Treatment', ascending=False).iloc[:5,:]

In [ ]:
# Under Treatment
fig = go.Figure()
fig.add_trace( go.Funnel(
    name = highest_ut_ratio.iloc[0,:].name,
    orientation='v',
    x = highest_ut_ratio.columns,
    y = highest_ut_ratio.iloc[0,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_ut_ratio.iloc[1,:].name,
    orientation = "v",
    x = highest_ut_ratio.columns,
    y = highest_ut_ratio.iloc[1,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_ut_ratio.iloc[2,:].name,
    orientation = "v",
    x = highest_ut_ratio.columns,
    y = highest_ut_ratio.iloc[2,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_ut_ratio.iloc[3,:].name,
    orientation = "v",
    x = highest_ut_ratio.columns,
    y = highest_ut_ratio.iloc[3,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_ut_ratio.iloc[4,:].name,
    orientation = "v", 
    x = highest_ut_ratio.columns,
    y = highest_ut_ratio.iloc[4,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.update_layout(height=700, title='Highest countries in Under Treatment case ratio')
fig.show()

# Recovered 
fig = go.Figure()
fig.add_trace( go.Funnel(
    name = highest_recovered_ratio.iloc[0,:].name,
    orientation='v',
    x = highest_recovered_ratio.columns,
    y = highest_recovered_ratio.iloc[0,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_recovered_ratio.iloc[1,:].name,
    orientation = "v",
    x = highest_recovered_ratio.columns,
    y = highest_recovered_ratio.iloc[1,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_recovered_ratio.iloc[2,:].name,
    orientation = "v",
    x = highest_recovered_ratio.columns,
    y = highest_recovered_ratio.iloc[2,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_recovered_ratio.iloc[3,:].name,
    orientation = "v",
    x = highest_recovered_ratio.columns,
    y = highest_recovered_ratio.iloc[3,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_recovered_ratio.iloc[4,:].name,
    orientation = "v",
    x = highest_recovered_ratio.columns,
    y = highest_recovered_ratio.iloc[4,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.update_layout(height=700, title='Highest countries in RECOVERED cases ratio')
fig.show()


# Deaths 
fig = go.Figure()
fig.add_trace( go.Funnel(
    name = highest_death_ratio.iloc[0,:].name,
    orientation='v',
    x = highest_death_ratio.columns,
    y = highest_death_ratio.iloc[0,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_death_ratio.iloc[1,:].name,
    orientation = "v",
    x = highest_death_ratio.columns,
    y = highest_death_ratio.iloc[1,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_death_ratio.iloc[2,:].name,
    orientation = "v",
    x = highest_death_ratio.columns,
    y = highest_death_ratio.iloc[2,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_death_ratio.iloc[3,:].name,
    orientation = "v",
    x = highest_death_ratio.columns,
    y = highest_death_ratio.iloc[3,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.add_trace(go.Funnel(
    name = highest_death_ratio.iloc[4,:].name,
    orientation = "v",
    x = highest_death_ratio.columns,
    y = highest_death_ratio.iloc[4,:],
    textposition = "auto", hoverinfo='x+y',
    textinfo = 'percent total'))

fig.update_layout(height=700, title='Highest countries in DEATH cases ratio')
fig.show()

# To Be Continued ...

Thank you. I hope that was a clear analysis of COVID-19 outbreak. Also, I hope that we do not need to analyze this dataset anymore. 
And don't forget to wash your hands everyone, Stay Healthy!

Please comment for any confusing part in the notebook, possible mistakes, criticisms, or advice. 